In [1]:
import torch
import torchvision 
import pandas as pd
import numpy as np
import argparse
from PIL import Image
import os
from tqdm import tqdm
import matplotlib.patches as patches
import matplotlib.pyplot as plt
from torch.utils import data
from torchvision import transforms, datasets, models
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
import time
import utils_ObjectDetection as utils
from torch.utils.data import BatchSampler,SequentialSampler,RandomSampler
import torch.nn as nn
from torch.nn.parallel import DistributedDataParallel as DDP
import math 
import ITB_RECALL_PRECISION_F1 as rp_f1

In [2]:
def make_prediction(model, img, threshold):
    model.eval()
    preds = model(img)
    for id in range(len(preds)) :
        idx_list = []

        for idx, score in enumerate(preds[id]['scores']) :
            
            if score > threshold : 
                idx_list.append(idx)

        preds[id]['boxes'] = preds[id]['boxes'][idx_list]
        preds[id]['labels'] = preds[id]['labels'][idx_list]
        preds[id]['scores'] = preds[id]['scores'][idx_list]

    return preds

def collate_fn(batch):
    return tuple(zip(*batch))


def get_transform():
    custom_transforms = []
    custom_transforms.append(torchvision.transforms.ToTensor())
    return torchvision.transforms.Compose(custom_transforms)


def get_model_instance_segmentation(num_classes,train_layer):
    # load an instance segmentation model pre-trained pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True,trainable_backbone_layers=train_layer)
    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    return model

def plot_image_from_output(img, annotation):
    
    img = img.cpu().permute(1,2,0)
    
    fig,ax = plt.subplots(figsize=(20, 10))
    ax.imshow(img)
    
    for idx in range(len(annotation["boxes"])):
        xmin, ymin, xmax, ymax = annotation["boxes"][idx]

        if annotation['labels'][idx] == 1 :
            rect = patches.Rectangle((xmin,ymin),(xmax-xmin),(ymax-ymin),linewidth=1,edgecolor='b',facecolor='none')
        
        elif annotation['labels'][idx] == 2 :
            
            rect = patches.Rectangle((xmin,ymin),(xmax-xmin),(ymax-ymin),linewidth=1,edgecolor='g',facecolor='none')
            
        elif annotation['labels'][idx] == 3 :
        
            rect = patches.Rectangle((xmin,ymin),(xmax-xmin),(ymax-ymin),linewidth=1,edgecolor='orange',facecolor='none')
        else:
            rect = patches.Rectangle((xmin,ymin),(xmax-xmin),(ymax-ymin),linewidth=1,edgecolor='r',facecolor='none')

        ax.add_patch(rect)

    return plt


In [3]:

class simpleDataset(object):


    def __init__(self, dataset,resize,color, img_foler_path,transforms=None):
    #             self.root = root
            self.transforms = transforms
            self.adnoc = dataset
            self.ids = dataset.index
            self. filenames=dataset['path'].to_list()
            self.resize =resize
            self.color=color
            self.img_foler_path=img_foler_path
            
    def __getitem__(self, index):
        # Own coco file
        adnoc_df = self.adnoc
        # Image ID ,폴더에서 index번째 
        img_id = self.ids[index]

       # List: get annotation id from coco, index번째 annotation가져오기 [[1,2,3,4],[5,6,7,8]]
        annotation = adnoc_df['box'][img_id]


        # open the input image, 흑백=L , 단색=1
        img= Image.open(str(self.img_foler_path)+str(adnoc_df.loc[img_id]['path'])).convert(self.color)
        img= img.resize((int(img.width / self.resize), int(img.height / self.resize)))
      
        # number of objects in the image
        num_objs = len(annotation)

        # Bounding boxes for objects
        # In coco format, bbox = [xmin, ymin, width, height]
        # In pytorch, the input should be [xmin, ymin, xmax, ymax]
        boxes = []
        areas = []
        label = []
        for i in range(num_objs):

            xmin = annotation[i][0]
            ymin = annotation[i][1]
            xmax = xmin + annotation[i][2]
            ymax = ymin + annotation[i][3]
            l=annotation[i][4]
            area=annotation[i][2]*annotation[i][3]

            boxes.append([xmin/self.resize, ymin/self.resize, xmax/self.resize, ymax/self.resize])
          
            label.append(l)
            areas.append(area)

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        #areas= box size = width * height
        areas = torch.as_tensor(areas, dtype=torch.float32)

        # Labels (In my case, I only one class: target class or background)
        labels = torch.as_tensor(label, dtype=torch.int64)

        # Tensorise img_id
        img_id = torch.tensor([img_id])

        # Iscrowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        # Annotation is in dictionary format
        my_annotation = {}
        my_annotation["boxes"] = boxes
        my_annotation["labels"] = labels
        my_annotation["image_id"] = img_id
        my_annotation["area"] = areas
        my_annotation["iscrowd"] = iscrowd

        if self.transforms is not None:
            img = self.transforms(img)

        return img, my_annotation

    def __len__(self):
        return len(self.ids)



    # the total number of samples (optional)
    def __len__(self):
        return len(self.filenames)

    


In [4]:
def result_RPA(test_data_loader,model,param,iteration):
    
    model = get_model_instance_segmentation(4,5)
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    model.to(device)
    model.load_state_dict(torch.load(param))
    model.eval()


    confi=[0.5]
    RECALL=[]
    PRECISION=[]
    F1_SCORE=[]
    for c in confi: 
        labels = []
        preds_adj_all = []
        annot_all = []
        print('confidence=',c)
        for im, annot in tqdm(test_data_loader, position = 0, leave = True):
            im = list(img.to(device) for img in im)

   
            for t in annot:
                labels += t['labels']

            with torch.no_grad():
                preds_adj = make_prediction(model, im, c)
                preds_adj = [{k: v.to(torch.device('cpu')) for k, v in t.items()} for t in preds_adj]
                preds_adj_all.append(preds_adj)
                annot_all.append(annot)

        #precision=0.2
        iou_threds=0.5
        pr_f1=rp_f1.Precision_Recall_F1(iou_threds,preds_adj_all,annot_all)
      
        
        recall=pr_f1.RECALL()
        precision=pr_f1.PRECISION()
        f1_score=pr_f1.F1_SCORE()
      

        RECALL.append(recall)
        PRECISION.append(precision)
        F1_SCORE.append(f1_score)
    
    
    print(RECALL,'\n',PRECISION,'\n',F1_SCORE)
    result=pd.DataFrame(confi,columns=['confidence'])
    result['iteration']=iteration
    result['recall']=[RECALL]
    result['precision']=[PRECISION]
    result['f1_form']=[F1_SCORE]
    
    
    return result

In [5]:
def random_sampling(train,batch):
   
    sampled=train.sample(n=batch,random_state=42)
    train_rest=train.drop(sampled.index)
        
    return sampled,train_rest


In [6]:
def data_load(data):
 
    # create own Dataset
    dataset = simpleDataset(dataset=data,
                                  resize=4,
                                  color='L',
                                  img_foler_path='D:/OBJ_PAPER/Data/itb_p_500/',
                                  transforms=get_transform())



    data_loader = torch.utils.data.DataLoader(dataset,
                                          batch_size=1,
                                          shuffle=False,
                                          num_workers=0,
                                          collate_fn=collate_fn)

    return data_loader

In [7]:
data_path='D:/OBJ_PAPER/Data/3_fold_cv/'

train1=pd.read_pickle(data_path+'train1.pkl')
train2=pd.read_pickle(data_path+'train2.pkl')
train3=pd.read_pickle(data_path+'train3.pkl')

valid1=pd.read_pickle(data_path+'valid1.pkl')
valid2=pd.read_pickle(data_path+'valid2.pkl')
valid3=pd.read_pickle(data_path+'valid3.pkl')

test=pd.read_pickle(data_path+'test.pkl')

# Random sampling

In [8]:

def TL_train_Random(Total,Batch,train,valid,test,Train_param_path,RESULT_path,Sampled_img_path):
    
    for total in Total:
        print(total)

        num_classes = 4
        num_epochs = 100
        train_layer=5
        batch=Batch
        patience=10

        model = get_model_instance_segmentation(num_classes,train_layer)
        device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

        # move model to the right device
        model.to(device)
        model.load_state_dict(torch.load('D:/OBJ_PAPER/model_param/model_fasterrcnn_bach5_p10_50_param.pt'))

        # parameters
        params = [p for p in model.parameters() if p.requires_grad]
        optimizer = torch.optim.Adam(params, lr=0.0001)

        iteration = 0    

        RESULT=pd.DataFrame()
        Sampled_img=pd.DataFrame()

        #모델 저장 
        train_param_path=Train_param_path.format(total)

        #data reload
        train_data_loader=data_load(train)
        valid_data_loader=data_load(valid)
        test_data_loader=data_load(test)



        while len(Sampled_img)<total:

            if len(Sampled_img)+batch>total:
                batch=total-len(Sampled_img)

            sampled,train_rest=random_sampling(train,batch)
            Sampled_img=Sampled_img.append(sampled,ignore_index=True)
            sample_data_loader=data_load(Sampled_img)
            
            Sampled_img['iter']=iteration

            #sample 제외한 나머지 train 데이터
            train=train_rest

            iteration += 1

            model.train()
            not_save_count=0
            st = time.time()

            #평균 loss
            avg_train_loss=[]
            avg_valid_loss=[]

            for epoch in range(num_epochs):

                # 모델이 학습되는 동안 trainning loss를 track
                train_losses = []
                # 모델이 학습되는 동안 validation loss를 track
                valid_losses = []
                
                st = time.time()
                for imgs, annotations in sample_data_loader:

                    imgs = list(img.to(device) for img in imgs)
                    annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
                    loss_dict = model(imgs, annotations)
                    #batch size=10, 10개 loss 각각 도출
                    losses = sum(loss for loss in loss_dict.values())
                    train_losses.append(losses.item())       
                    optimizer.zero_grad()
                    losses.backward()
                    optimizer.step()

                 #이미지 한장당 평균 loss
                avg_train_loss.append(np.mean(train_losses).round(5))

                #validation, early_stop, save weights
                with torch.no_grad():

                    for im, annot in valid_data_loader:
                        im = list(img.to(device) for img in im)
                        annot = [{k: v.to(device) for k, v in t.items()} for t in annot]
                        val_loss_dict = model(im, annot)
                        val_losses = sum(val_loss for val_loss in val_loss_dict.values())
                        valid_losses.append(val_losses.item())

                    epoch_val_loss=np.mean(valid_losses).round(5)
                    avg_valid_loss.append(epoch_val_loss)  

                fi = time.time()     
                print('epoch:',epoch,'train_loss: ',np.mean(train_losses).round(5),'validation loss : ',np.mean(valid_losses).round(5),'time',fi-st)


                min_val_loss=np.min(avg_valid_loss)
                if min_val_loss>=epoch_val_loss:

                    torch.save(model.state_dict(),train_param_path)
                    not_save_count=0
                    print('epoch:',epoch,'save model')

                else:
                    not_save_count+=1
                    model.load_state_dict(torch.load(train_param_path))
                    if not_save_count>=patience:
                        print('no more training')
                        break



            fi = time.time()     
            print('iteration:',iteration,'train_loss: ',np.mean(train_losses).round(5),'time',fi-st)
            print('sample num:',len(Sampled_img))
            model.load_state_dict(torch.load(train_param_path))
            
            result_batch=result_RPA(test_data_loader,model,train_param_path,iteration)
            RESULT=RESULT.append(result_batch,ignore_index=True)

        RESULT.to_csv(RESULT_path.format(total),index=False)   
        Sampled_img.to_csv(Sampled_img_path.format(total),index=False)   
        
        return RESULT

In [9]:
Total=[348]
Batch=30
train=train1
valid=valid1
test=test
Train_param_path='D:/OBJ_PAPER/model_param/FRCN_itb_random1_batch30_total{}_param.pt'
RESULT_path='D:/OBJ_PAPER/result/FRCN_itb_random1_batch30_total{}_result.csv'
Sampled_img_path='D:/OBJ_PAPER/result/FRCN_itb_random1_batch30_total{}_sampled.csv'

TL_train_Random(Total,Batch,train,valid,test,Train_param_path,RESULT_path,Sampled_img_path)

348
epoch: 0 train_loss:  0.2475 validation loss :  0.23735 time 14.424647331237793
epoch: 0 save model
epoch: 1 train_loss:  0.1579 validation loss :  0.24508 time 9.294165849685669
epoch: 2 train_loss:  0.13379 validation loss :  0.25864 time 9.277958393096924
epoch: 3 train_loss:  0.12885 validation loss :  0.25232 time 9.281796932220459
epoch: 4 train_loss:  0.13061 validation loss :  0.25683 time 9.270211935043335
epoch: 5 train_loss:  0.13224 validation loss :  0.2763 time 9.292159080505371
epoch: 6 train_loss:  0.12996 validation loss :  0.26678 time 9.30854320526123
epoch: 7 train_loss:  0.13261 validation loss :  0.26575 time 9.252346277236938
epoch: 8 train_loss:  0.13319 validation loss :  0.27769 time 9.25153112411499
epoch: 9 train_loss:  0.13153 validation loss :  0.27885 time 9.330727815628052
epoch: 10 train_loss:  0.12981 validation loss :  0.27213 time 9.254356622695923
no more training
iteration: 1 train_loss:  0.12981 time 9.395301818847656
sample num: 30
confidence

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:09<00:00, 10.71it/s]


precision [0.23684, 0.81609, 0.62201]
recall [0.50943, 0.98611, 0.83333]
[[0.50943, 0.98611, 0.83333]] 
 [[0.23684, 0.81609, 0.62201]] 
 [[0.32335, 0.89308, 0.71233]]
epoch: 0 train_loss:  0.21145 validation loss :  0.23762 time 14.491578578948975
epoch: 0 save model
epoch: 1 train_loss:  0.18689 validation loss :  0.21155 time 14.253196477890015
epoch: 1 save model
epoch: 2 train_loss:  0.15151 validation loss :  0.19635 time 14.250759840011597
epoch: 2 save model
epoch: 3 train_loss:  0.12409 validation loss :  0.18869 time 14.264440298080444
epoch: 3 save model
epoch: 4 train_loss:  0.10867 validation loss :  0.19942 time 14.520627975463867
epoch: 5 train_loss:  0.10239 validation loss :  0.22365 time 14.300076484680176
epoch: 6 train_loss:  0.11673 validation loss :  0.19913 time 14.25854229927063
epoch: 7 train_loss:  0.11063 validation loss :  0.21187 time 14.225983381271362
epoch: 8 train_loss:  0.1187 validation loss :  0.20796 time 14.203076839447021
epoch: 9 train_loss:  0.10

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:08<00:00, 11.75it/s]


precision [0.58537, 0.84146, 0.86747]
recall [0.90566, 0.95833, 0.92308]
[[0.90566, 0.95833, 0.92308]] 
 [[0.58537, 0.84146, 0.86747]] 
 [[0.71111, 0.8961, 0.89441]]
epoch: 0 train_loss:  0.16966 validation loss :  0.42624 time 19.456435918807983
epoch: 0 save model
epoch: 1 train_loss:  0.22636 validation loss :  0.28981 time 19.454981327056885
epoch: 1 save model
epoch: 2 train_loss:  0.18045 validation loss :  0.18257 time 19.271204948425293
epoch: 2 save model
epoch: 3 train_loss:  0.13231 validation loss :  0.17273 time 19.230036973953247
epoch: 3 save model
epoch: 4 train_loss:  0.11778 validation loss :  0.18093 time 19.25792169570923
epoch: 5 train_loss:  0.11199 validation loss :  0.16535 time 19.25409960746765
epoch: 5 save model
epoch: 6 train_loss:  0.09204 validation loss :  0.16585 time 19.350342750549316
epoch: 7 train_loss:  0.09406 validation loss :  0.1742 time 19.405197143554688
epoch: 8 train_loss:  0.08702 validation loss :  0.17045 time 19.27405023574829
epoch: 9 

...

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:08<00:00, 11.59it/s]


precision [0.78947, 0.92208, 0.83146]
recall [0.84906, 0.98611, 0.94872]
[[0.84906, 0.98611, 0.94872]] 
 [[0.78947, 0.92208, 0.83146]] 
 [[0.81818, 0.95302, 0.88623]]
epoch: 0 train_loss:  0.06451 validation loss :  0.17308 time 62.35337543487549
epoch: 0 save model
epoch: 1 train_loss:  0.08341 validation loss :  0.12057 time 62.216511487960815
epoch: 1 save model
epoch: 2 train_loss:  0.0609 validation loss :  0.13094 time 62.09102010726929
epoch: 3 train_loss:  0.05763 validation loss :  0.11969 time 61.98756217956543
epoch: 3 save model
epoch: 4 train_loss:  0.05291 validation loss :  0.14221 time 62.131627559661865
epoch: 5 train_loss:  0.0509 validation loss :  0.14251 time 62.18280792236328
epoch: 6 train_loss:  0.05351 validation loss :  0.14776 time 62.063246726989746
epoch: 7 train_loss:  0.05358 validation loss :  0.12308 time 62.0254123210907
epoch: 8 train_loss:  0.056 validation loss :  0.14879 time 62.04355263710022
epoch: 9 train_loss:  0.05312 validation loss :  0.1409

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:08<00:00, 11.70it/s]


precision [0.87755, 0.94667, 0.76263]
recall [0.81132, 0.98611, 0.96795]
[[0.81132, 0.98611, 0.96795]] 
 [[0.87755, 0.94667, 0.76263]] 
 [[0.84314, 0.96599, 0.85311]]


,confidence,iteration,recall,precision,f1_form
0,0.5,1,"[[0.50943, 0.98611, 0.83333]]","[[0.23684, 0.81609, 0.62201]]","[[0.32335, 0.89308, 0.71233]]"
1,0.5,2,"[[0.90566, 0.95833, 0.92308]]","[[0.58537, 0.84146, 0.86747]]","[[0.71111, 0.8961, 0.89441]]"
2,0.5,3,"[[0.75472, 0.97222, 0.98077]]","[[0.58824, 0.93333, 0.66522]]","[[0.66116, 0.95238, 0.79275]]"
3,0.5,4,"[[0.88679, 0.98611, 0.94231]]","[[0.67143, 0.91026, 0.85965]]","[[0.76423, 0.94667, 0.89908]]"
4,0.5,5,"[[0.81132, 0.98611, 0.96795]]","[[0.82692, 0.83529, 0.78238]]","[[0.81905, 0.90446, 0.86533]]"
5,0.5,6,"[[0.83019, 0.98611, 0.96795]]","[[0.81481, 0.92208, 0.81622]]","[[0.82243, 0.95302, 0.88563]]"
6,0.5,7,"[[0.88679, 0.95833, 0.95513]]","[[0.52809, 0.98571, 0.87647]]","[[0.66197, 0.97183, 0.91411]]"
7,0.5,8,"[[0.83019, 0.98611, 0.97436]]","[[0.8, 0.86585, 0.87356]]","[[0.81482, 0.92208, 0.92121]]"
8,0.5,9,"[[0.90566, 0.98611, 0.98077]]","[[0.6, 0.84524, 0.80105]]","[[0.7218, 0.91026, 0.88185]]"
9,0.5,10,"[[0.84906, 0.98611, 0.96154]]","[[0.84906, 0.8875, 0.86705]]","[[0.84906, 0.93421, 0.91185]]"


In [ ]:
Total=[100,200,300,343]
Batch=30
train=train2
valid=valid2
test=test
train_param_path='D:/OBJ_PAPER/model_param/FRCN_itb_random2_batch30_total{}_param.pt'
RESULT_path='D:/OBJ_PAPER/result/FRCN_random2_b30_total_{}_validation_p{}.csv'
Sampled_img_path='D:/OBJ_PAPER/result/FRCN_random2_b30_total_{}_validation_p{}_sampled.csv'

TL_train_Random(Total,Batch,train,valid,test,Train_param_path,RESULT_path,Sampled_img_path)

In [ ]:
Total=[100,200,300,343]
Batch=30
train=train3
valid=valid3
test=test
train_param_path='D:/OBJ_PAPER/model_param/FRCN_itb_random3_batch30_total{}_param.pt'
RESULT_path='D:/OBJ_PAPER/result/FRCN_random3_b30_total_{}_validation_p{}.csv'
Sampled_img_path='D:/OBJ_PAPER/result/FRCN_random3_b30_total_{}_validation_p{}_sampled.csv'

TL_train_Random(Total,Batch,train,valid,test,Train_param_path,RESULT_path,Sampled_img_path)